# Kansalliskirjaston ALTO XML:n tekstinlouhintaesimerkkejä (Digitalia-notebook)

Versiohistoria
* 0.2  30.9.2021 Aineiston lataaminen rajapinnan kautta, korjauksia koodiin ja käännöksiin (Erno Liukkonen) 
* 0.1  23.4.2019 Pieniä tekstikorjauksia
* 0.02 Digitalia-project, pieniä korjauksia
* 0.01 15.1.2019 (Mika Koistinen)


# Kiitokset



<table><tr><td>
<img src="https://blogs.helsinki.fi/digitalia/files/2018/10/sosiaali_fi_90p.jpg" style="height:100px;width:100%">
    </td><td>
<img src="https://blogs.helsinki.fi/digitalia/files/2018/10/fi_EU_rgb_90p.jpg" style="height:100px;width:100%">
    </td><td>
<img src="https://blogs.helsinki.fi/digitalia/files/2015/10/digitalia_pien_512.png" style="height:70px;width:100%">
    </td></td></tr></table>

# Aineiston lataaminen rajapinnan kautta

Aineiston lataaminen tapahtuu rajapintaa hyödyntäen. Suorita haluamasi aineiston haku https://digi.kansalliskirjasto.fi/search osoitteessa ja kopio hakutulokset sisältävän sivun osoite alla olevaan searchResultsUrl muuttujaan (oletuksena nyt https://digi.kansalliskirjasto.fi/search?startDate=1870-10-01&endDate=1870-12-31&pages=1-50&title=1457-4721&formats=NEWSPAPER).

bindingSearchQuery funktio suorittaa haettujen aineistojen tietojen lataamisen rajapinnan kautta. Funktiolle annetaan parametrinä hakutulokset sisältävän sivun osoite.

In [ ]:
import urllib
from urllib.error import URLError, HTTPError
from socket import timeout
import simplejson as json

searchResultsUrl = "https://digi.kansalliskirjasto.fi/search?startDate=1870-10-01&endDate=1870-12-31&pages=1-50&title=1457-4721&formats=NEWSPAPER"

request_headers = {
"User-Agent": "Notebook dam-rajapinta",
"Referer": "Notebook dam-rajapinta",
"Connection": "keep-alive" 
}

pageImageTemplate = ""
altoXmlTemplate = ""
altoTxtTemplate = ""

def bindingSearchQuery(digiResultsUrl):
    
    
    global pageImageTemplate
    global altoXmlTemplate
    global altoTxtTemplate

    parameters = digiResultsUrl[digiResultsUrl.index('?'):]

    currentRows = []
    result = ""
    noMoreResults = False
    isFirstSearch = True

    digiBindingSearchURL = 'https://digi.kansalliskirjasto.fi/api/dam/binding-search' + parameters

    while noMoreResults == False:
    
      req = urllib.request.Request(digiBindingSearchURL, headers=request_headers)
      try:
        response = urllib.request.urlopen(req, timeout=30)
        responseResult = response.read()
        result = json.loads(responseResult)

        pageImageTemplate = result["pageImageTemplate"]
        altoXmlTemplate = result["altoXmlTemplate"]
        altoTxtTemplate = result["altoTxtTemplate"]

        if len(result["rows"]) != 0:
          currentRows = currentRows + result["rows"]
        else:
          noMoreResults = True

        if isFirstSearch == True:
          digiBindingSearchURL = 'https://digi.kansalliskirjasto.fi/api/dam/binding-search/' + result["scrollId"]
          isFirstSearch = False

      except HTTPError as e:
        content = e.read()
      except ConnectionError as e:
        print("Ei yhteyttä palvelimeen!")
      except URLError as e:
        print("Ei yhteyttä palvelimeen!")
      except TimeoutError as e:
        print("Ei yhteyttä palvelimeen!")
      except timeout:
        print("Ei yhteyttä palvelimeen!")

    result["rows"] = currentRows

    return result

Seuraavaksi suoritetaan varsinainen ALTO- ja teksti-tiedostojen lataus urlretrieve funktion avulla, funktioon tehtävien kutsujen rakentamisessa käytetään bindingSearchQuery funktion kautta saatuja tietoja. Tiedostot ladataan oletuksena downloads-hakemistoon, joka sijaitsee samassa hakemistossa mistä notebook käynnistettiin.

In [ ]:
import sys
import os
import urllib
from urllib.error import URLError, HTTPError
from socket import timeout

def urlretrieve(url, localfile):
    
    global searchResultsUrl
    
    try:
      req = urllib.request.Request(url, headers=request_headers)
      f = urllib.request.urlopen(req, timeout=30)

      # Open our local file for writing
      with open(localfile, "wb") as fl:
          fl.write(f.read())

    #handle errors
    except HTTPError as e:
      print("Ei yhteyttä palvelimeen!")
      return 0 
    except URLError as e:
      print("Ei yhteyttä palvelimeen!")
      return 0
    except TimeoutError as e:
      print("Ei yhteyttä palvelimeen!")
      return 0
    except timeout:
      print("Ei yhteyttä palvelimeen!")
      return 0

    return 1

notebookPath =  os.path.dirname(os.path.realpath("__file__"))

result = bindingSearchQuery(searchResultsUrl)

rows = result["rows"]

isFirstResultRow = True
firstResultAlto = ""
firstResultPath = ""
 
resultCounter = 1

for row in rows:
    bindingTitle = row["bindingTitle"]
    bindingId = str(row["bindingId"])
    pageNumber =  str(row["pageNumber"])
    baseUrl = row["baseUrl"]
    
    altoUrl  = baseUrl + altoXmlTemplate.replace("{{page}}", str(pageNumber))
    txtUrl  = baseUrl + altoTxtTemplate.replace("{{page}}", str(pageNumber))
    
    altoPath = notebookPath + "/downloads/" + bindingTitle + "_" + bindingId + "/alto/"
    txtPath = notebookPath + "/downloads/" + bindingTitle + "_" + bindingId + "/txt/"

    if not os.path.exists(altoPath):
        os.makedirs(altoPath)
    
    if not os.path.exists(txtPath):
        os.makedirs(txtPath)
        
    downloadPathAlto = altoPath + bindingTitle + "_" + bindingId + "_page_" + pageNumber + ".xml"
    downloadPathTxt = txtPath + bindingTitle + "_" + bindingId + "_page_" + pageNumber + ".txt"
    
    print("Ladataan tulosta: " + str(resultCounter) + "/" + str(len(rows)))
    
    urlretrieve(altoUrl, downloadPathAlto)
    urlretrieve(txtUrl, downloadPathTxt)
    
    resultCounter += 1
    
    if isFirstResultRow:
        firstResultAlto = downloadPathAlto
        firstResultPath = altoPath
        isFirstResultRow = False

# Ladataan yksi tiedosto DataFrameen ym.

In [ ]:
import os

global firstResultPath

#Ensimmäisen ladatun niteen ALTO-hakemiston polku, joka rajapinnan kautta ladattiin
path = firstResultPath 

print(path)


In [ ]:
#tällä voi katsoa mitä tiedostoja kansiossa:
sorted(os.listdir(path)) 

In [ ]:
from bs4 import BeautifulSoup as bs

global firstResultAlto

#valitaan ensimmäinen ladattu alto-tiedosto
filename = firstResultAlto
    

In [ ]:
xmlsoup=bs(open(filename,"r"),"lxml")
xmldata=str(xmlsoup)


In [ ]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd

In [ ]:
#muokkaa XML-rakenteen pandas.DataFrame:ksi
#alkuperäinen funktio löytyy:
#http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/
class XML2DataFrame:

    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

In [ ]:
#luodaan tietorakenne 
xml2df = XML2DataFrame(xmldata)
xml_dataframe = xml2df.process_data()
#näyttää tietorakenteen kappaleet
xml_dataframe.columns



In [ ]:
#luodaan tiedoston teksteistä pandas.DataFrame
strings=xmlsoup.find_all(["string","hyp"])


In [ ]:
strings

In [ ]:
#luodaan xmldata2 jossa stringit erikseen rakenteen sisällä
xmldata2='<?xml version="1.0" encoding="UTF-8"?><html><body>'
for s in strings:
    xmldata2=xmldata2+str(s)+"\n"
xmldata2=xmldata2+'</body></html>'

In [ ]:
xmldata2

In [ ]:
tree = ET.XML(xmldata2)


In [ ]:
#luodaan pd.Dataframe ja tulostetaan se
data = []
for el in tree.iterfind('./*'):
    for i in el.iterfind('*'):
        data.append(dict(i.items()))
df = pd.DataFrame(data)        
df

In [ ]:
#tulostetaan koko teksti (yhdistää huomioi hyppartit eli rivillä tavuviivaan katkeavat)
contents=""
for w in df['content']: 
   #if w!="-":        
   contents=contents+w+" "
   #cont=contents.replace("\n"," ")
contents2=contents.replace(" - ","")
print(contents2)     

In [ ]:
#luodaan sanatokenit
#NLTK
#pip install nltk
import nltk 
tokens=nltk.word_tokenize(contents2)
tokens[0:10]

# Word Frequencies /sanojen esiintymistiheydet

In [ ]:
#removing stopwords
from nltk.corpus import stopwords
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#sulkusanat
stop_words=set(stopwords.words('finnish'))
#print(stopwords[0:10],len(stopwords))
#filtered_sentence = [w for w in content if not w in stop_words]

#sulkusanat poistetaan
filtered_content=[]
for r in tokens:
    #print(r)
    if not r in stop_words:
        filtered_content.append(r)

#sulkusanat alle 1 merkin sanat ja yli 20 merkin sanat poistetaan
filtered_content2=[]
for r in tokens:
    #print(r)
    if not r in stop_words and len(r) <20 and len(r)>1: # and
        filtered_content2.append(r)

In [ ]:
#tulostetaan osa eritavoin esikäsitellyistä tokeneista
freqz=nltk.FreqDist(tokens)
freqz2=nltk.FreqDist(filtered_content)
freqz3=nltk.FreqDist(filtered_content2)

print("koko teksti")
print(freqz.most_common()[0:15])
print("stopwords poistettu")
print(freqz2.most_common()[0:15])

# WordClouds / sanapilvet 

In [ ]:
#löytyy täältä esimerkkiä 
#https://github.com/amueller/word_cloud
#!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


tokens_str = ''.join(tokens)
fig = plt.figure(figsize=(24, 14)) 
wordcloud=WordCloud().generate(tokens_str)
#wordcloud = WordCloud(max_font_size=50,min_font_size=5).generate(content_str)
plt.subplot(221)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("1 document (all tokens)")
plt.show()


fig = plt.figure(figsize=(24, 14)) 
content_str2 = ''.join(filtered_content)
wordcloud2=WordCloud().generate(content_str2)
#wordcloud2 = WordCloud(max_font_size=40,min_font_size=5).generate(content_str2)
plt.subplot(222)
plt.imshow(wordcloud2, interpolation="bilinear")
plt.title("1 document (removed stopwords)")


plt.axis("off")
plt.show()

print("sanojen määrä:",len(tokens_str),len(content_str2))#,len(content_str3))
#content_str2




# 2. Useampi tiedosto kansiosta


In [ ]:
# loading all xml files from the folders / ladataan kaikki tiedostot kansiosta
import os
global firstResultPath

#Ensimmäisen ladatun niteen ALTO-hakemiston polku, joka rajapinnan kautta ladattiin
path = firstResultPath 
files=os.listdir(path)
#sorted(os.listdir(path))


In [ ]:
#!huom tässä menee hetkinen
strings2=[]
for file1 in files:
    filename=path+file1
    xmlsoup=bs(open(filename,"r"),"lxml")
    strings2.extend(xmlsoup.find_all("string"))

print("sanojen määrä kaikki sivut kansiossa:",len(strings2))
    
    

In [ ]:
content22=[stri2['content'].lower() for stri2 in strings2[0:]]
len(content22)

In [ ]:
#poistetaan alle 2 merkin pituiset sanat
content2=[stri2 for stri2 in content22 if len(stri2)>1]
len(content2)

In [ ]:
#poistetaan ohitettavat sanat (stopwords)
filtered_content3=[]
for r in content2:
    #print(r)
    if not r in stop_words:
        filtered_content2.append(r)
len(filtered_content2)

In [ ]:
freqz3=nltk.FreqDist(content2)
freqz3.most_common()[0:15]

freqz4=nltk.FreqDist(filtered_content2)

print(freqz3.most_common()[0:10])
print(freqz4.most_common()[0:10])



# WordClouds / sanapilvet (kaikki hakemistossa olevat sivut)

In [ ]:
content_str2 = ' '.join(content2)



fig = plt.figure(figsize=(16, 10))
plt.subplot(223)
wordcloud2 = WordCloud(max_font_size=50,min_font_size=5).generate(content_str2)

plt.imshow(wordcloud2, interpolation="bilinear")
plt.title("all "+str(len(files))+" documents, and "+str(len(content2))+" words, from folder")

plt.axis("off")


content_str3 = ' '.join(filtered_content2)

plt.subplot(224)
wordcloud3 = WordCloud(max_font_size=50,min_font_size=5).generate(content_str3)
plt.imshow(wordcloud3, interpolation="bilinear")

plt.title("all "+str(len(files))+" documents, and "+str(len(filtered_content2))+" words, from folder (removed stopwords)")

plt.axis("off")
plt.show()

# Topic Modeling / Aihemallinnus

In [ ]:
#based on 
#https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/
#read all docs into the corpora
import os
files=os.listdir(path)
FILES=[]
doc_complete=[]
for ind in range(0,len(files)):
    filename=path+files[ind]
    FILES.append(filename)
    xmlsoup=bs(open(filename,"r"),"xml")
    strings=xmlsoup.find_all("String")
    strings2=[]
    for str1 in strings:
        strings2.append(str1['CONTENT'].lower())
    doc_complete.append(strings2)

len(doc_complete),len(content2),len(filtered_content2)



In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('finnish'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    doc2=[word for word in doc if len(word)>3]       
    stop_free = " ".join([i for i in doc2 if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

    
doc_clean = [clean(doc).split() for doc in doc_complete]

In [ ]:
#doc_clean=docs2#filtered_content2#map(unicode,filtered_content2)
# Importing Gensim
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(doc_clean )
#dictionary
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]



In [ ]:
# Creating the object for LDA model using gensim library

Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=100, id2word = dictionary, passes=50)

In [ ]:
#Results
print(ldamodel.print_topics(num_topics=100, num_words=3))


In [ ]:
print(ldamodel.print_topics(num_topics=10, num_words=5))

In [ ]:
len(doc_complete),len(doc_clean)

TOPIC MODELING2 https://de.dariah.eu/tatom/topic_model_python.html

In [ ]:
#FILES
FILES2=[]
import os
global firstResultPath

#Ensimmäisen ladatun niteen ALTO-hakemiston polku, joka rajapinnan kautta ladattiin, vaihdetaan hakemiston nimestä "alto" osa "txt" osaan. 
path = firstResultPath.replace("alto","txt") 
for file in os.listdir(path):
    if file.endswith(".txt"):
        FILES2.append("".join([path,file]))


In [ ]:
############

import numpy as np
#FILES[0:4]
stop = list(set(stopwords.words('finnish')))

import sklearn.feature_extraction.text as text
vectorizer = text.CountVectorizer(input='filename', stop_words=stop, min_df=1)
dtm = vectorizer.fit_transform(FILES2).toarray()
vocab = np.array(vectorizer.get_feature_names())


In [ ]:
dtm.shape,len(vocab)

In [ ]:
from sklearn import decomposition
num_topics = 10
num_top_words = 10
clf = decomposition.NMF(n_components=num_topics, random_state=1)



In [ ]:
doctopic = clf.fit_transform(dtm)

In [ ]:
topic_words = []
for topic in clf.components_:
   word_idx = np.argsort(topic)[::-1][0:num_top_words]
   topic_words.append([vocab[i] for i in word_idx])

In [ ]:
doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)

In [ ]:
novel_names = []

# turn this into an array so we can use NumPy functions
#novel_names = np.asarray(novel_names)
novel_names=np.asarray(FILES2)
doctopic_orig = doctopic.copy()

# use method described in preprocessing section
num_groups = len(set(novel_names))
doctopic_grouped = np.zeros((num_groups, num_topics))

for i, name in enumerate(sorted(set(novel_names))):
        doctopic_grouped[i, :] = np.mean(doctopic[novel_names == name, :], axis=0)

doctopic = doctopic_grouped

In [ ]:
novels = sorted(set(novel_names))

print("Top NMF topics in...")


for i in range(len(doctopic)):
        top_topics = np.argsort(doctopic[i,:])[::-1][0:3]
        top_topics_str = ' '.join(str(t) for t in top_topics)
        print("{}: {}".format(novels[i], top_topics_str))
    


In [ ]:
for t in range(len(topic_words)):
   print("Topic {}: {}".format(t, ' '.join(topic_words[t][:15])))